In [11]:
import math
import prime_gen

In [12]:
# =================================================
# ================== CRYPTPO ======================
# =================================================

In [13]:
# https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Modular_inverse
# pour calculer l'inver modulaire
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)
def modinv(a, m):
    """
    inverse de a modulo m 
    """
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [14]:
# provient de http://python.jpvweb.com/python/mesrecettespython/doku.php?id=est_premier
# test de MillerRabin
# =============================================================================
petitspremiers = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,
    79,83,89,97,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,
    179,181,191,193,197,199,211,223,227,229,233,239,241,251,257,263,
    269,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,
    367,373,379,383,389,397,401,409,419,421,431,433,439,443,449,457,
    461,463,467,479,487,491,499,503,509,521,523,541,547,557,563,569,
    571,577,587,593,599,601,607,613,617,619,631,641,643,647,653,659,
    661,673,677,683,691,701,709,719,727,733,739,743,751,757,761,769,
    773,787,797,809,811,821,823,827,829,839,853,857,859,863,877,881,
    883,887,907,911,919,929,937,941,947,953,967,971,977,983,991,997,
    1009,1013,1019,1021]
# =============================================================================
def lpowmod(a, b, n):
    """exponentiation modulaire: calcule (a**b)%n"""
    r = 1
    while b>0:
        if b&1==0:
            b = b>>1
        else:
            r = (r*a)%n
            b = (b-1)>>1
        a = (a*a)%n
    return r
 
# =============================================================================
#  Test de primalité probabiliste de Miller-Rabin
def _millerRabin(a, n):
    """Ne pas appeler directement (fonction utilitaire). Appeler millerRabin(n, k=20)"""
    # trouver s et d pour transformer n-1 en (2**s)*d
    d = n - 1
    s = 0
    while d % 2 == 0:
        d >>= 1
        s += 1
 
    # calculer l'exponentiation modulaire (a**d)%n
    apow = lpowmod(a,d,n) # =(a**d)%n
 
    # si (a**d) % n ==1 => n est probablement 1er
    if apow == 1:
        return True
 
    for r in range(0,s):
        # si a**(d*(2**r)) % n == (n-1) => n est probablement 1er
        if lpowmod(a,d,n) == n-1:
            return True
        d *= 2
 
    return False
 
# ========================
def millerRabin(n, k=20):
    """Test de primalité probabiliste de Miller-Rabin"""
    global petitspremiers

    # éliminer le cas des petits nombres <=1024
    if n<=1024:
        if n in petitspremiers:
            return True
        else:
            return False
    
    # éliminer le cas des nombres pairs qui ne peuvent pas être 1ers!
    if n & 1 == 0:
        return False
 
    # recommencer le test k fois: seul les nb ayant réussi k fois seront True
    for repete in range(0, k):
        # trouver un nombre au hasard entre 1 et n-1 (bornes inclues)
        a = random.randint(1, n-1)
        # si le test echoue une seule fois => n est composé
        if not _millerRabin(a, n):
            return False
    # n a réussi les k tests => il est probablement 1er
    return True

# ======================== par moi
def premier_nb_alea(depart):
    """
    donnet le premier nombre premier à partir du depart
    """
    p = 0
    if (depart%2 ==0):
        depart = depart + 1 # pour avoir nb impair
    while (True):
        t = millerRabin(depart)
        if t == True:
            p = depart
            return p
        else:
            depart = depart+2
            continue

In [15]:
# code de http://python.jpvweb.com/python/mesrecettespython/doku.php?id=restes_chinois
# reste chinois 
def pgcd(*n):
    """Calcul du 'Plus Grand Commun Diviseur' de n (>=2) valeurs entières (Euclide)"""
    def _pgcd(a,b):
        while b: a, b = b, a%b
        return a
    p = _pgcd(n[0], n[1])
    for x in n[2:]:
        p = _pgcd(p, x)
    return p    
    
def pgcde(a, b):
    """ pgcd étendu avec les 2 coefficients de bézout u et v
        Entrée : a, b entiers
        Sorties : r = pgcd(a,b) et u, v entiers tels que a*u + b*v = r
    """
    r, u, v = a, 1, 0
    rp, up, vp = b, 0, 1
    while rp != 0:
        q = r//rp
        rs, us, vs = r, u, v
        r, u, v = rp, up, vp
        rp, up, vp = (rs - q*rp), (us - q*up), (vs - q*vp)
    return (r, u, v)

def ppcm(a,b):
    """ppcm(a,b): calcul du 'Plus Petit Commun Multiple' entre 2 nombres entiers a et b"""
    if (a==0) or (b==0):
        return 0
    else:
        return (a*b)//pgcd(a,b)

def rchinois(arg):
    """Calcul des restes chinois pour n (>=2) équations modulaires
    Argument = liste de n couples: arg = [[a1,m1],[a2,m2],...,[an,mn]]
    Calcul de x qui résout: x%m1==a1,  x%m2==a2 ... x%mn==an
    Avec m1, m2, ..., mn entiers quelconques (>0 ou <0) mais <>0
    Condition d'existence de solution: (ai,aj)%pgcd(mi,mj)==0 (i<>j)
    Retour de x accompagné de son modulo
    """
 
    a1, m1 = arg[0]
    for a2, m2 in arg[1:]:        
        r, y1, y2 = pgcde(m2, m1) # r=pgcd; y1 et y2 =coef de Bézout de m2 et m1
        if (a1-a2) % r != 0:
            return None, None  # il n'y a pas de solution => on renvoie None
        a1 = ((a1*m2*y1 + a2*m1*y2) % (m1*m2))//r
        m1 = ppcm(m1,m2)
    return a1 % m1, m1  # renvoi de x et de son modulo, 

In [16]:
def isMultiple(a, n):
    """
    true si a multiple de n
    """
    return (a%n==0)

def premier_sur(a,b):
    """
    nb premier sur entre a et b exclu
    """
    qpp = a

    while True:
        print(a)
        while True:                      # 1. Choisir q'' non multiple de 3, 5 7 et 11.
            qpp = random.randint(1,a)
            if not isMultiple(qpp,3) and not isMultiple(qpp,5) and not isMultiple(qpp,7)and not isMultiple(qpp,11):
                q = (6 *qpp)+3           # 2. q <--- 6q'' + 3
                break
                
        if millerRabin(q):               # 4. p <--- 12q'' + 7
            
            p = (12 *qpp) +7
            if millerRabin(p):
                return (q,p)
            else:                         # 5. Si p n'est pas premier, aller en 6. Sinon c'est gagné.
                q = (6*qpp) +5
                if millerRabin(q):
                    p = (12*qpp) +11
                    if millerRabin(p):
                        return (q,p)
                    else:
                        continue
                else:
                    break
                
        else:  # 3. Si q n'est pas premier, aller en 6.
            q = (6*qpp) +5
            if millerRabin(q):
                p = (12*qpp) +11
                if millerRabin(p):
                    return (q,p)
                else:
                    continue
            else:
                continue

In [17]:
from random import randrange, getrandbits
def is_prime(n, k=128):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True
def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p
def generate_prime_number(length=1024):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in          bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 128):
        p = generate_prime_candidate(length)
    return p

In [18]:
def bsgs(g, h, p):
    '''
    Solve for x in h = g^x mod p given a prime p.
    If p is not prime, you shouldn't use BSGS anyway.
    '''
    N = int(sqrt(p - 1))  # phi(p) is p-1 if p is prime

    # Store hashmap of g^{1...m} (mod p). Baby step.
    tbl = {pow(g, i, p): i for i in range(N)}

    # Precompute via Fermat's Little Theorem
    c = pow(g, N * (p - 2), p)

    # Search for an equivalence in the table. Giant step.
    for j in range(N):
        y = (h * pow(c, j, p)) % p
        if y in tbl:
            return j * N + tbl[y]

    # Solution not found
    return None

In [19]:
# =================================================
# ===================== HACK ======================
# =================================================

In [22]:
def key_format(key):
    args = ['openssl', 'pkey', '-pubin', '-in', key, '-text']
        
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    
    stdout, stderr = pipeline.communicate()

    error_message = stderr.decode()

    if error_message != '':
        raise OpensslError(error_message)
    
    return stdout

def maniere_dure(key):
    args = ['openssl', 'asn1parse', '-in', key, '-i', '-strparse','19']
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    stdout, stderr = pipeline.communicate()
    error_message = stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)    
    return stdout

In [23]:
def factorisation_bete(n):
    for p in prime_gen.primes():

        if n%p==0:
            res = n//p
            return (res,p)

In [21]:
def F(x,c,N):
    return ((x*x)+c) %N

def rho(N):
    c=1
    slow=2 
    fast=F(slow,c,N)
    slowp = F(slow,c,N)
    fastp=F(F(slow,c,N),c,N)
    
    while(True):

        g=sympy.gcd( (fast-slow)*(fastp-slowp), N)
        if g==N:
            c+=1
            continue
        if g != 1:
            return g # gagné
        
        slow=F(slow,c,N)
        fast=F(F(fast,c,N),c,N)
        slowp = F(slow,c,N)
        fastp=F(F(slow,c,N),c,N)
def rho2(N):
    c=1
    trap = 0
    nexttrap = 1
    slow = 2
    fast = slow
    i = 0

    while(True):
        fast = F(fast,c,N)


        g = math.gcd(fast-slow, N)
        if g==N:
            c+=1
            continue
        if g != 1:
            return g # gagné

        if i == nexttrap:
            trap =nexttrap
            slow=fast
            nexttrap =2*trap
            print(i)
        i =i + 1